In [57]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm

import cvxpy as cp

(CVXPY) Apr 19 03:40:35 PM: Encountered unexpected exception importing solver GLPK:
ImportError("cannot import name 'glpk' from 'cvxopt' (/home/artem/Study/SDAFE/venv/lib/python3.11/site-packages/cvxopt/__init__.py)")
(CVXPY) Apr 19 03:40:35 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("cannot import name 'glpk' from 'cvxopt' (/home/artem/Study/SDAFE/venv/lib/python3.11/site-packages/cvxopt/__init__.py)")


#### Exercise 1

In [18]:
r1 = 2.3
r2 = 4.5
sigma1 = np.sqrt(6)
sigma2 = np.sqrt(11)
rho = 0.17

(a) 
$$
w r_1 + (1-w) r_2 = r_t
$$
gives us
$$w = \frac{r_t - r_2}{r_1 - r_2}$$

In [38]:
rt = 3
w = (rt - r2) / (r1 - r2)
w

0.6818181818181818

In [39]:
w * r1 + (1 - w) * r2

3.0

(b)
$$
\mathrm{Var}[wr_1 + (1-w)r_2] = w^2 \sigma_1^2 + (1-w)^2 \sigma_2^2 + 2w(1-w)\rho\sigma_1\sigma_2 = \sigma^2,
$$
gives us
$$w_{1,2} = \frac{-2\sigma_2(\rho \sigma_1 - \sigma_2) \pm \sqrt{D}}{2(\sigma_1^2 + \sigma_2^2 - 2 \rho \sigma_1 \sigma_2)},$$
where
$$D = 4 \sigma_2^2 (\rho \sigma_1 - \sigma_2)^2 - 4(\sigma_2^2 - \sigma_t^2)(\sigma_1^2 + \sigma_2^2 - 2 \rho \sigma_1 \sigma_2)$$

In [40]:
sigmat = np.sqrt(5.5)

In [41]:
D = 4 * sigma2 ** 2 * (rho * sigma1 - sigma2) ** 2 - 4 * (sigma2 ** 2 - sigmat ** 2) * (sigma1 ** 2 + sigma2 ** 2 - 2 * rho * sigma1 * sigma2)
w = (-2 * sigma2 * (rho * sigma1 - sigma2) + np.array([1, -1]) * np.sqrt(D)) / 2 / (sigma1 ** 2 + sigma2 ** 2 - 2 * rho * sigma1 * sigma2)
w

array([0.94039993, 0.41077726])

In [42]:
def var(w, sigma1, sigma2, rho):
    return np.sqrt(w ** 2 * sigma1 ** 2 + (1 - w) ** 2 * sigma2 ** 2 + 2 * w * (1 - w) * rho * sigma1 * sigma2)

In [35]:
var(w, sigma1, sigma2, rho)

array([2.34520788, 2.34520788])

#### Exercise 2

Let $r_t$ and $\sigma_t$ be the expected return and the standard deviation of return on the tangency portfolio, $w_c$ the weight allocated to the risk-free asset, and $\sigma_*$ the target standard deviation.

Then
$$w_c r_f + (1 - w_c) r_t = r_*$$
and
$$(1 - w_c)^2 \sigma_t^2 = \sigma_*^2,$$
which gives us two solutions:
$$w_c = 1 \pm \frac{\sigma_*}{\sigma_t}.$$

Consider the solution $$w_c = 1 + \frac{\sigma_*}{\sigma_t},$$
then the resulting portfolio return
$$r_* = \left( 1 + \frac{\sigma_*}{\sigma_t} \right) r_f - \frac{\sigma_*}{\sigma_t} r_t = r_f + \frac{\sigma_*}{\sigma_t}(r_f - r_t).$$
Normally, $r_t > r_f$ which then in this case implies $r_* < r_f$. There is no point constructing a portfolio with a return below risk-free, hence we can ignore this solution.

This leaves us with
$$w_c = 1 - \frac{\sigma_*}{\sigma_t}.$$

The $i$-th asset weight in the overall portfolio is then given by $\frac{\sigma_*}{\sigma_t} w_i$, where $w_i$ is the $i$-th asset weight in the tangency portfolio.


In [43]:
w1 = 0.65
w2 = 0.35
rt = 5
sigmat = 7
rf = 1.5
sigma_target = 5

In [44]:
wc = 1 - sigma_target / sigmat

The weight of the risk-free asset is:

In [45]:
wc

0.2857142857142857

The weight of asset C:

In [46]:
(1 - wc) * w1

0.4642857142857143

The weight of asset D:

In [48]:
(1 - wc) * w2

0.25

The weights add up to 1:

In [50]:
wc + (1 - wc) * w1 + (1 - wc) * w2

1.0

#### Exercise 3

(a-b) The weight of the $j$-th stock in a portfolio is given by
$$w_j = \frac{P_j n_j}{\sum_{i=1}^N P_i n_i}.$$

In [52]:
p = np.array([75, 115])
n = np.array([300, 100])

The weights of stocks in the portfolio are:

In [53]:
p * n / np.sum(p * n)

array([0.66176471, 0.33823529])

#### Exercise 4

Let prices of individual assets be $p_i$ and the quantities of those assets in the portfolio $n_i$.

Then the weights of assets are given by
$$w_i = \frac{p_i n_i}{\sum_{j=1}^N p_j n_j},$$
the net returns by
$$r_i = \frac{p'_i}{p_i} - 1,$$
and the gross returns by
$$R_i = \frac{p'_i}{p_i},$$
where $p_i$ is the price of the asset at time $t-1$ and $p'_i$ is the price of the asset at time $t$. From this we have
$$p'_i - p_i = r_i p_i.$$

The value of the entire portfolio at time $t-1$ is given by
$$P = \sum_{i=1}^N p_i n_i$$
and at time $t$ by
$$P' = \sum_{i=1}^N p'_i n_i.$$

The net return on the portfolio is
$$
r = \frac{P'}{P'} - 1
= \frac{\sum_{i=1}^N p'_i n_i}{\sum_{i=1}^N p_i n_i} - 1 
= \frac{\sum_{i=1}^N p'_i n_i - \sum_{i=1}^N p_i n_i}{\sum_{i=1}^N p_i n_i}
= \frac{\sum_{i=1}^N (p'_i - p_i) n_i}{\sum_{i=1}^N p_i n_i}
= \frac{\sum_{i=1}^N r_i p_i n_i}{\sum_{i=1}^N p_i n_i}
= \sum_{i=1}^N r_i \frac{p_i n_i}{\sum_{j=1}^N p_j n_j}
= \sum_{i=1}^N r_i w_i.
$$

The gross return on the portfolio is
$$
R = \frac{P'}{P'} = r + 1 = \sum_{i=1}^N r_i w_i + 1 = \sum_{i=1}^N r_i w_i + \sum_{i=1}^N w_i = \sum_{i=1}^N (r_i + 1) w_i = \sum_{i=1}^N R_i w_i.
$$

However for log returns:
$$
\log R = \log \left( \sum_{i=1}^N R_i w_i \right) \neq \sum_{i=1}^N w_i \log R_i.$$

#### Exercise 5

In [87]:
mu = np.array([0.0032, 0.0074])
sigma = np.array([
    [0.0170, 0.0059],
    [0.0059, 0.025],
])
mu_b = np.array([0.0047, 0.0065])
sigma_b = np.array([
    [0.0125, 0.0058],
    [0.0058, 0.023],
])
mu_target = 0.005

(a)

In [88]:
w = cp.Variable(2)
risk = cp.quad_form(w, sigma_b)
prob = cp.Problem(cp.Minimize(risk), [w.T @ mu_b == mu_target, cp.sum(w) == 1])
prob.solve()
w.value

array([0.83333333, 0.16666667])

The solution provided by the authors (https://people.orie.cornell.edu/davidr/SDAFE2/Solutions/chapter16.pdf) does not appear to be correct:

In [ ]:
w_sol = np.array([0.5714, 0.4286])

In [ ]:
w_sol.T @ mu_b

0.00547148

In [101]:
w_sol.T @ sigma_b @ w_sol

0.011147141244

In [102]:
w_sol.T @ mu

0.00500012

In [103]:
w_sol.T @ sigma @ w_sol

0.013032758392

(b)

In [64]:
risk.value

0.010930555555555551

(c) The actual portfolio mean return is:

In [65]:
w.value.T @ mu

0.003900000000000004

and the variance is:

In [66]:
w.value.T @ sigma @ w.value

0.01413888888888888

#### Exercise 6

In [67]:
mu = np.array([0.001, 0.0015])
n = np.array([200, 100])
p = np.array([100, 125])
sd = np.array([0.03, 0.04])
rho = 0.35

In [81]:
w = n * p / np.sum(n * p)

The correlation matrix of the stock returns is:

In [78]:
corr = np.array([
    [1, rho],
    [rho, 1]
])
corr

array([[1.  , 0.35],
       [0.35, 1.  ]])

In [79]:
sigma = np.diag(sd) @ corr @ np.diag(sd)

The return on the portfolio:

In [82]:
w.T @ mu

0.0011923076923076924

The standard deviation of the portfolio return:

In [85]:
np.sqrt(w.T @ sigma @ w)

0.027862723501961284